In [ ]:
import torch.nn.functional as F
import torch.nn as nn

class DigitClassifier(nn.Module):
  def __init__(self):
    super().__init__()
    self.matrix_multiply_hidden1 = nn.Linear(64,18) # Trying two layer and 10 nodes for each layer
    self.matrix_multiply_hidden2 = nn.Linear(18,18)
    self.matrix_multiply_output = nn.Linear(18,10)
  def forward(self,x):
    x = self.matrix_multiply_hidden1(x)
    x = F.relu(x)
    x = self.matrix_multiply_hidden2(x)
    x = F.relu(x)
    x = self.matrix_multiply_output(x)
    return x

digit_classifier_model = DigitClassifier()


In [ ]:
import torch
from torchvision import transforms
transform_funcs = transforms.Compose([
    transforms.ToTensor(), # Convert data into a tensor(vector)
    transforms.Normalize((0.5),(0.5)) #Normalize data using mean and standard deviation for 3 channel using x_normalized = (x-mean)/std
])

In [ ]:
from torchvision.datasets import MNIST  # Load MNISt Dataset and apply transformation
mnist_dataset = MNIST("data", download=True, train=True, transform= transform_funcs)

100%|██████████| 9.91M/9.91M [00:00<00:00, 137MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 16.5MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 25.5MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.09MB/s]


In [ ]:
# Use DataLoader Class to partition the data into traning and validation sets
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
# create training and validation split && creating samples
split= int(0.8 * len(mnist_dataset))
data_index_list = list(range(len(mnist_dataset)))
training_index , validation_index = data_index_list[:split], data_index_list[split:]
training_sampler_class = SubsetRandomSampler(training_index)
validation_sampler_class = SubsetRandomSampler(validation_index)

# creating training and validation data loader objects
training_loader = DataLoader (mnist_dataset, batch_size = 256, sampler=training_sampler_class)
validation_loader = DataLoader(mnist_dataset, batch_size=256, sampler= validation_sampler_class)

In [ ]:
import matplotlib.pyplot as plt
import cv2
import numpy as np
NUM_EPOCHS = 200
resized_image = []

# 宣告超參數
learning_rate = 0.01
weight_decay = 1e-6

from torch import optim
cross_entripy_loss_function = torch.nn.CrossEntropyLoss()
model_optimizer = optim.Adam(digit_classifier_model.parameters(),
                            lr=learning_rate, weight_decay=weight_decay)

for epoch in range(1, NUM_EPOCHS):
  training_loss = []
  validation_loss = []## 訓練迴圈
  for input_data, target_prediction in training_loader:
    rimages = []
    for i in range(input_data.shape[0]):# 將圖像從 28x28 調整為 8x8
      image = input_data[i][0]
      rimage = cv2.resize(image.numpy(), dsize=(8, 8), interpolation=cv2.INTER_CUBIC)
      rimage = rimage.reshape(1, rimage.shape[0], rimage.shape[1])
      rimages.append(rimage)

    resized_image_batch = np.stack(rimages, axis=0)
    resize_tensor = torch.tensor(resized_image_batch)

    input_data = resize_tensor.view(-1, 64)
    model_optimizer.zero_grad()# 前向傳播
    model_output = digit_classifier_model(input_data)# 損失計算
    model_loss = cross_entripy_loss_function(model_output, target_prediction)# 反向傳播
    model_loss.backward()# 權重優化
    model_optimizer.step()
    training_loss.append(model_loss.item())## 準備模型進行評估
  digit_classifier_model.eval()

  # 評估迴圈
  for input_data, target_prediction in validation_loader:
    rimages = []
    for i in range(input_data.shape[0]):
      image = input_data[i][0]
      rimage = cv2.resize(image.numpy(), dsize=(8, 8), interpolation=cv2.INTER_CUBIC)
      rimage = rimage.reshape(1, rimage.shape[0], rimage.shape[1])
      rimages.append(rimage)

    resized_image_batch = np.stack(rimages, axis=0)
    resize_tensor = torch.tensor(resized_image_batch)

    input_data = resize_tensor.view(-1, 64)
    model_output = digit_classifier_model(input_data)
    model_loss = cross_entripy_loss_function(model_output, target_prediction)
    validation_loss.append(model_loss.item())

  print(np.mean(training_loss))


1.0444748655278633


KeyboardInterrupt: 

In [ ]:
!pip install onnx
!pip install tensorflow-addons
!git clone https://github.com/onnx/onnx-tensorflow.git && cd onnx-tensorflow && pip install -e .
!pip install torchvision
torch.save(digit_classifier_model.state_dict(), 'mnist.pth')

In [ ]:
!pip install netron
!pip install onnxruntime

In [ ]:
import torch
import onnx
x = torch.randn(1, 64)
torch_out = digit_classifier_model(x)
torch.onnx.export(digit_classifier_model, x, "digit_classifier.onnx", export_params=True, opset_version=10, input_names = ['input'], output_names = ['output'])

onnx_model = onnx.load("digit_classifier.onnx")
onnx.checker.check_model(onnx_model)

import onnxruntime
ort_session = onnxruntime.InferenceSession("digit_classifier.onnx")

def to_numpy(tensor):
  return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

ort_inputs = {ort_session.get_inputs()[0].name: to_numpy(x)}
ort_outs = ort_session.run(None, ort_inputs)

np.testing.assert_allclose(to_numpy(torch_out), ort_outs[0], rtol=1e-03, atol=1e-05)


In [ ]:
import numpy as np
for param_tensor in digit_classifier_model.state_dict():
  print(param_tensor, "\t", digit_classifier_model.state_dict()[param_tensor].size())
  values = list(digit_classifier_model.state_dict()[param_tensor].numpy().flatten())
  filename = param_tensor + '.txt'
  txt_file = open(filename, 'w')
  for v in values:
    txt_file.write(str(v)+'\n')
txt_file.close()

In [ ]:
# @title
from google.colab import files
files.download('matrix_multiply_hidden.weight.txt')
files.download('matrix_multiply_hidden.bias.txt')
files.download('matrix_multiply_output.weight.txt')
files.download('matrix_multiply_output.bias.txt')

In [ ]:
files.download('digit_classifier.onnx')
files.download('mnist.pth')

In [ ]:
!ls

sample_data
